Towards 3D Deep Learning for neuropsychiatry: predicting Autism diagnosis using an interpretable Deep Learning pipeline applied to minimally processed structural MRI data, Melanie Garcia, Clare Kelly. medRxiv 2022.10.18.22281196; doi: https://doi.org/10.1101/2022.10.18.22281196

Github: https://github.com/garciaml/Autism-3D-CNN-brain-sMRI?tab=readme-ov-file

In [ ]:
# Activate Virtual Environment and Install Requirements
#!python3 -m venv ../pretrainedresnet2
#!source ../pretrainedresnet2/bin/activate
#!python3 -m ipykernel install --user --name=pretrainedresnet2 --display-name "Python (pretrainedresnet2)"
#Switch to notebook/virtual environment kernel
#!pip install -r requirements.txt # install requirements text in new environment
#!pip install "torchio>=0.19.0"
#!pip install monai
#!pip install tensorboard
#!pip install torchsummary

In [1]:
import os
import shutil
import pandas as pd

# EDA

In [2]:
def copy_with_structure(source_folder, destination_folder):
    """Copies the contents of source_folder to destination_folder, maintaining the directory structure."""

    for item in os.listdir(source_folder):
        source_path = os.path.join(source_folder, item)
        destination_path = os.path.join(destination_folder, item)

        if os.path.isfile(source_path):
            shutil.copy2(source_path, destination_path)  # copy file with metadata
        elif os.path.isdir(source_path):
             shutil.copytree(source_path, destination_path, dirs_exist_ok=True) # copy directory and its contents
        else:
            print(f"Skipping {source_path}, not a file or directory.")

In [3]:
def count_folders(path):
    folder_count = 0
    for item in os.listdir(path):
        if os.path.isdir(os.path.join(path, item)):
            folder_count += 1
    return folder_count

In [4]:
def count_files(directory_path):
    """Counts the number of files in a directory.

    Args:
        directory_path: The path to the directory.

    Returns:
        The number of files in the directory, or -1 if the directory does not exist.
    """
    if not os.path.exists(directory_path):
        return -1
    
    file_count = 0
    for item in os.listdir(directory_path):
        item_path = os.path.join(directory_path, item)
        if os.path.isfile(item_path):
            file_count += 1
    return file_count

In [14]:
def count_autism_labels(filepath):
    # Load the TSV file
    try:
        df = pd.read_csv(filepath, sep='\t')
    except Exception as e:
        print(f"Error reading file: {e}")
        return

    # Check required columns
    required_columns = {'label', 'dataset'}
    if not required_columns.issubset(df.columns):
        print(f"Missing required columns. Found columns: {df.columns.tolist()}")
        return

    # Debug: print unique values
    print("Unique labels:", df['label'].unique())
    print("Unique dataset entries:", df['dataset'].unique())

    # Convert numeric labels to string labels
    label_map = {1: 'Autism', 0: 'No Autism'}
    df['label'] = df['label'].map(label_map)

    if df['label'].isnull().any():
        print("Warning: Some labels could not be mapped (not 0 or 1).")

    # Count labels per dataset split
    for split in ['train', 'val', 'test']:
        subset = df[df['dataset'] == split]
        if subset.empty:
            print(f"No data for split: {split}")
            continue

        autism_count = (subset['label'] == 'Autism').sum()
        no_autism_count = (subset['label'] == 'No Autism').sum()

        print(f"{split.upper()} SET:")
        print(f"  Autism: {autism_count}")
        print(f"  No Autism: {no_autism_count}\n")

# Model Training - check if can reporduce results from paper (train to 42 epochs with similar split)

#Copy ABIDE1 and ABIDE2 images to ABIDE_COMBINED folder in JustBrain_Data folder
source_folder1 = "JustBrain_Data/ABIDE1"
source_folder2 = "JustBrain_Data/ABIDE2"
destination_folder = "JustBrain_Data/ABIDE_COMBINED"

copy_with_structure(source_folder1, destination_folder)
copy_with_structure(source_folder2, destination_folder)


#Combine ABIDEI and ABIDE2 tsv files
participants1_tsv = pd.read_csv('JustBrain_Data/ABIDE1/participants.tsv', sep="\t", dtype=str)
participants1_tsv['participant_id'] = participants1_tsv['participant_id'].astype(str).str.zfill(7)
participants2_tsv = pd.read_csv('JustBrain_Data/ABIDE2/participants.tsv', sep="\t", dtype=str)
participants_ABIDE = pd.concat([participants1_tsv,participants2_tsv])
participants_ABIDE.to_csv('JustBrain_Data/ABIDE_COMBINED/participants.tsv', sep='\t', index=False, header=True)

#Check
print(count_folders(source_folder1) + count_folders(source_folder2))
print(count_folders(destination_folder))
print(participants_ABIDE.shape[0]) # - looks good!


#Copy ABIDE1 and ABIDE2 images to ABIDE_COMBINED folder in Preprocessed_Data folder
source_folder1 = "Preprocessed_Data/ABIDE1"
source_folder2 = "Preprocessed_Data/ABIDE2"
destination_folder = "Preprocessed_Data/ABIDE_COMBINED"

copy_with_structure(source_folder1, destination_folder)
copy_with_structure(source_folder2, destination_folder)

#Check
test_folder = count_files("Preprocessed_Data/ABIDE_COMBINED/test")
train_folder = count_files("Preprocessed_Data/ABIDE_COMBINED/train")
val_folder = count_files("Preprocessed_Data/ABIDE_COMBINED/val")
print(test_folder + train_folder + val_folder)

In [16]:
#count_autism_labels('JustBrain_Data/ABIDE1/participants.tsv')
#count_autism_labels('JustBrain_Data/ABIDE2/participants.tsv')
count_autism_labels('JustBrain_Data/ABIDE_COMBINED/participants.tsv')

Unique labels: [1 0]
Unique dataset entries: ['train' 'val' 'test']
TRAIN SET:
  Autism: 799
  No Autism: 899

VAL SET:
  Autism: 142
  No Autism: 157

TEST SET:
  Autism: 59
  No Autism: 56



# Train Model

In [2]:
# Start training from scratch
# batch size = 4 (6 too large)
# learning rate = 0.0001
# seed = 42

In [ ]:
!python ../Autism-3D-CNN-brain-sMRI/train_medicalnet.py 'JustBrain_Data/ABIDE_COMBINED' 'Preprocessed_Data/ABIDE_COMBINED' './outputs/Resnet50/ABIDE_Combined' '../Autism-3D-CNN-brain-sMRI/resnet_training/resnet_10.pth'

Loading pretrained model weights from Resnet50 (backbone)
----------
epoch 1
1/424, train_loss: 0.5798
2/424, train_loss: 0.5303
3/424, train_loss: 1.1646
4/424, train_loss: 1.7541
5/424, train_loss: 0.9332
6/424, train_loss: 0.6520
7/424, train_loss: 2.0708
8/424, train_loss: 1.2679
9/424, train_loss: 0.7920
10/424, train_loss: 0.6910


In [ ]:
# Resume training from saved model
#!python ../Autism-3D-CNN-brain-sMRI/train_medicalnet.py 'JustBrain_Data/ABIDE_COMBINED' 'Preprocessed_Data/ABIDE_COMBINED' './outputs/Resnet50/ABIDE_Combined' 'outputs/Resnet50/ABIDE_Combined/checkpoint_1.pth' --resume

In [1]:
with open("outputs/Resnet50/ABIDE_Combined/training.log", "r") as infile, open("outputs/Resnet50/ABIDE_Combined/filtered.log", "w") as outfile:
    for line in infile:
        if line.startswith("epoch ") and "average loss" in line:
            outfile.write(line)
        #elif line.startswith("Epoch time duration:"):
            #outfile.write(line)
        elif line.startswith("current epoch:"):
            outfile.write(line)

## Test

In [4]:
participants_tsv = pd.read_csv('JustBrain_Data/ABIDE_COMBINED/participants.tsv', sep="\t", dtype=str)
participants_tsv.rename(columns={"participant_id" : "SUB_ID"}, inplace=True)
participants_tsv = participants_tsv[participants_tsv.dataset == 'test']
participants_tsv.to_csv('outputs/Resnet50/ABIDE_Combined/test/subjects.csv', index = False)

In [3]:
# Predictions - 18 epochs (best accuracy) - overfit - predicts 0 for everything
!python ../Autism-3D-CNN-brain-sMRI/predict_medicalnet_subids.py 'Preprocessed_Data/ABIDE_COMBINED/test' 'outputs/Resnet50/ABIDE_Combined/test/subjects.csv' './outputs/Resnet50/ABIDE_Combined/checkpoint_18.pth' './outputs/Resnet50/ABIDE_Combined/test'

evaluation metric: 0.5130434782608696


## Val

In [6]:
participants_tsv = pd.read_csv('JustBrain_Data/ABIDE_COMBINED/participants.tsv', sep="\t", dtype=str)
participants_tsv.rename(columns={"participant_id" : "SUB_ID"}, inplace=True)
participants_tsv = participants_tsv[participants_tsv.dataset == 'val']
participants_tsv.to_csv('outputs/Resnet50/ABIDE_Combined/validation/subjects.csv', index = False)

In [ ]:
# Predictions - 36 epochs (predictions all 0) - overfit
!python ../Autism-3D-CNN-brain-sMRI/predict_medicalnet_subids.py 'Preprocessed_Data/ABIDE_COMBINED/val' 'outputs/Resnet50/ABIDE_Combined/validation/subjects.csv' './outputs/Resnet50/ABIDE_Combined/checkpoint_36.pth' './outputs/Resnet50/ABIDE_Combined/validation'

## Train

In [ ]:
participants_tsv = pd.read_csv('JustBrain_Data/ABIDE_COMBINED/participants.tsv', sep="\t", dtype=str)
participants_tsv.rename(columns={"participant_id" : "SUB_ID"}, inplace=True)
participants_tsv = participants_tsv[participants_tsv.dataset == 'train']
participants_tsv.to_csv('outputs/Resnet50/ABIDE_Combined/train/subjects.csv', index = False)

In [10]:
# Predictions - 36 epochs (predictions all 0) - overfit
!python ../Autism-3D-CNN-brain-sMRI/predict_medicalnet_subids.py 'Preprocessed_Data/ABIDE_COMBINED/train' 'outputs/Resnet50/ABIDE_Combined/train/subjects.csv' './outputs/Resnet50/ABIDE_Combined/checkpoint_36.pth' './outputs/Resnet50/ABIDE_Combined/train'

/sfs/gpfs/tardis/home/ejh2wy/Autism-3D-CNN-brain-sMRI/resnet2.py:174: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')
/sfs/gpfs/tardis/home/ejh2wy/CHMCorr_Autism_Research/../Autism-3D-CNN-brain-sMRI/predict_medicalnet_subids.py:104: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_g